# First eval

This jupyter notebook is for trying out what research I have read and my ideas in this [Notion](https://www.notion.so/Model-evaluation-170d3cfe027d80889de3cfbb35b531a4).

I played around with a Mistral Large model to build a mathematics chatbot. It is simple and I prompt engineered it. I might change the model to a smaller and cheaper one, but I still want to use a model that is considerd good enough. My plan is to use three small open source models to evaluate the mahtematics model. I'll also use different models for the mathematics chatbot and for the evaluation models to get realible results.

I'll use this [dataset](https://github.com/google-deepmind/mathematics_dataset?tab=readme-ov-file) as a test data set. I'll select 50 mathematics questiosn from the dataset. Then I'll ask the mathematicsbot the same questions. 
Then I'll let: 
- First a Mistral Large model compare the answers. Same model as in the mathematicsbot but different prompt engineering. 
    - This is not a good way to evaluate models, but I wish to try it. Espeacilly when it is only 50 questions it is manageble for me to compare the answers.
- Then I'll let three smaller open-source models compare the answers.
- After that I'll add a fourth model to compare the answers of the three open-open source model to give a final evaluation about the mathematics chat bot.

In [61]:
!pip3 install mistralai
!pip3 install python-dotenv
!pip3 install mathematics_dataset
!pip3 install pip install sympy==1.6
!pip3 install numpy==1.24.3


[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement install (from versions: none)
ERROR: No matching distribution found for install

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


I've used the following prompt for the Mistral Large 2.1 model
"Your task is to help solve math problems. Generate three answers that helps solving the problem but doesn't give the solution. When generating the 4th answer to the problem you can give the solution. When you don't know the answer say that you don't know the answer. Write the mathematical part of the answer in latex."

In [2]:
import os
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()

api_key = os.environ["MISTRAL_API_KEY"]
agent_id = os.environ["MISTRAL_AGENT_ID"]

client = Mistral(api_key=api_key)

chat_response = client.agents.complete(
    agent_id= agent_id,
    messages=[
        {
            "role": "user",
            "content": """Given the matrix:
                            A = [
                                [4, 1, -2],
                                [1, 3,  0],
                                [-2, 0, 2]
                            ]

                            Tasks:
                            1. Compute the eigenvalues of A.
                            2. Compute the eigenvectors of A for each eigenvalue.
                            3. Verify that for each eigenpair (λ, v), the equation A * v = λ * v holds.
                            4. If possible, diagonalize A by finding matrices P (eigenvectors) and D (diagonal matrix of eigenvalues) such that A = P * D * P⁻¹.
                            """
        },
    ],
)
print(chat_response.choices[0].message.content)

1. To compute the eigenvalues of the matrix $A$, you need to solve the characteristic equation $\det(A - \lambda I) = 0$, where $I$ is the identity matrix. This involves finding the roots of the characteristic polynomial.

2. For each eigenvalue $\lambda$ found in step 1, solve the equation $(A - \lambda I)v = 0$ to find the corresponding eigenvectors $v$. This involves solving a homogeneous system of linear equations for each eigenvalue.

3. To verify that for each eigenpair $(\lambda, v)$, the equation $A * v = \lambda * v$ holds, substitute the eigenvalue $\lambda$ and the corresponding eigenvector $v$ into the equation and check if the left-hand side equals the right-hand side.

4. To diagonalize the matrix $A$, we need to form the matrix $P$ whose columns are the eigenvectors of $A$, and the diagonal matrix $D$ whose diagonal elements are the eigenvalues of $A$. Then, we can check if $A = P * D * P^{-1}$.

   For the given matrix $A = \begin{bmatrix}
   4 & 1 & -2 \\
   1 & 3 & 0 

I have used the [mathematics_dataset](https://github.com/google-deepmind/mathematics_dataset) from Google DeepMind that will work as a test set. The main issue is that I don't know if the Mistral Large model that is used for the mathematics model have been trained on this data. There is a great chans for that. 

Never the less, I made a json file called questions_and_answers of the [mathematics_dataset](https://github.com/google-deepmind/mathematics_dataset) so that I can easily separet the questions from the answers. Ideally I would run all the questions through the mathematics model, but due to budget questions that won't be possible. I'll choose around 50 questions from the dataset. 

In [107]:
import json
import re

# Assuming 'output' is fetched from your command
output = !python -m mathematics_dataset.generate

# Initialize variables for parsing
cleaned_output = []
current_qa = {"question": "", "answer": ""}
in_answer = False

# Regular expression pattern to remove color codes
color_code_pattern = re.compile(r'\x1B(?:[@-Z\\-_]|\[[0-?]*[ -/]*[@-~])')

# Regular expression to detect log messages
log_message_pattern = re.compile(r'W\d{4}\s\d{2}:\d{2}:\d{2}.\d{6}\s\d+\sgenerate\.py:\d+\]\s.*')

# Regular expression to detect start of new questions - now with lookahead
new_question_pattern = re.compile(r'(?=^(?:Let |Solve |Suppose |Calculate |What |Round |Give |Factor |Determine |Find |Put together |Add |Subtract |Work out |Evaluate |In base \d+))', re.MULTILINE)


def extract_answer(text):
    """Extract the complete mathematical answer"""
    # Remove any color codes and extra whitespace
    text = color_code_pattern.sub('', text.strip())
    
    # If this is a mathematical expression (contains variables, operators, etc.)
    if re.search(r'[a-zA-Z\*\+\-\^\(\)]', text):
        # Split on obvious new question markers
        parts = re.split(r'\s+(?=Let|Solve|Suppose|Calculate|What|Round|Give|Factor|Determine|Find)', text)
        return parts[0].strip()
    
    # For numerical answers, split on additional calculations
    else:
        parts = text.split(' Total of ')
        parts = parts[0].split(' Calculate ')
        parts = parts[0].split(' What is ')
        return parts[0].strip()

def save_current_qa():
    global current_qa, cleaned_output, in_answer
    if current_qa["question"] and current_qa["answer"]:
        question = current_qa["question"].strip()
        answer = extract_answer(current_qa["answer"])
        
        if answer:  # Only save if we have an answer
            # Remove any trailing keywords that might have been included
            answer = re.sub(r'\s+(?:Let|Solve|Suppose|Calculate|What|Round|Give|Factor|Determine|Find)$', '', answer)
            
            cleaned_output.append({
                "question": color_code_pattern.sub('', question).strip(),
                "answer": color_code_pattern.sub('', answer).strip()
            })
        
        current_qa = {"question": "", "answer": ""}
        in_answer = False


# Process the output line by line
for line in output:
    # Skip log messages
    if log_message_pattern.match(line):
        continue
        
    line = line.strip()
    if not line:  # Skip empty lines
        continue

    # Skip subheadings
    if len(line) > 0 and " " not in line and not (line.endswith(".") or line.endswith("?")):
        continue

    # Check if this line starts a new question
    if new_question_pattern.match(line):
        # Save the previous Q&A if it exists
        save_current_qa()
        
        # Start new question
        if "  " in line:
            q, a = line.split("  ", 1)
            current_qa["question"] = q.strip()
            current_qa["answer"] = a.strip()
            in_answer = True
        else:
            current_qa["question"] = line.strip()
    else:
        # Continue with current question or answer
        if "  " in line:
            if not in_answer:
                q, a = line.split("  ", 1)
                current_qa["question"] += " " + q.strip()
                current_qa["answer"] = a.strip()
                in_answer = True
            else:
                current_qa["answer"] += " " + line.strip()
        else:
            if in_answer:
                current_qa["answer"] += " " + line.strip()
            else:
                current_qa["question"] += " " + line.strip()

# Save the last Q&A pair
save_current_qa()

# Save to JSON file
with open('questions_and_answers.json', 'w') as json_file:
    json.dump(cleaned_output, json_file, indent=4)

print("Output saved to questions_and_answers.json")

Output saved to questions_and_answers.json


In [108]:
import json

# Load the JSON data from the file
with open('questions_and_answers.json', 'r') as json_file:
    data = json.load(json_file)

# Extract only the questions
questions = [entry["question"] for entry in data]

# Print or save the questions as needed
print(questions)

# Optionally, you can save the questions to a new JSON file
with open('questions.json', 'w') as json_file:
    json.dump(questions, json_file, indent=4)

print("Questions saved to questions.json")

['Solve -1497 = -9*s - 1506 for s.', 'Solve 625*a + 538 = 731*a - 1476 for a.', 'Solve 431*l + 6515 = -812 for l.', 'Solve 62 = 7*c + 97 for c.', 'Solve -1415113 = -24876*p + 898355 for p.', 'Solve 13 = 7*b + 6 for b.', 'Solve -274*h + 6 = -272*h for h.', 'Solve 0 = 166*v - 287841 + 289169 for v.', 'Solve 51*t + 616 - 1990 = 135*t + 894 for t.', 'Solve -g = 6*g + g for g.', 'Suppose 20 = -5*v + 5, -5*k - 5*v - 35 = 0. Let i = k + 5. Suppose 2*c = -0*c + 10. Solve 0 = c*d - 4 - i for d.', 'Let w be (-8)/(-36)*-3*-6. Suppose -w + 14 = q. Let s = -54 - -58. Solve d + s*d = -q for d.', 'Suppose 0 = 13*o - 760 - 930. Suppose o*b - 4 = 128*b. Let i = -11 - -15. Solve -b*r = -0 - i for r.', 'Let p be (4/6)/(2/33). Solve 5*x + 1 = p for x.', 'Let j be 5*-4*(-3)/b. Solve j*d - d = -16 for d.', 'Suppose 5*f - s = 371, 216 = 2*f - s + 70. Suppose -75 = -4*r - a, -7*r + f = -2*r + 5*a. Solve -8*n + r = -4*n for n.', 'Solve 3*g - g + w = v for g.', 'Suppose 5*f - 3 = 7. Suppose 0 = -5*l + 15. Suppo